**Objective :** This project aims to develop a deep learning model to identify whether an image contains a forest fire using a convolutional neural network (CNN). The model is trained using a dataset of forest fire images and evaluated for its accuracy and loss.

**Data Preprocessing and Augmentation :**

The project begins by importing necessary libraries, including TensorFlow and Keras for model creation and data manipulation. The dataset for training and testing the model is stored in Google Drive.

The ImageDataGenerator class from Keras is used to perform data augmentation on the training set. This includes:

*   Rescaling the pixel values to the range [0, 1].

*   Shear transformations and zoom to increase variability.

*   Horizontal flipping for further augmentation.



**Model Architecture :**

The model uses a VGG16 pre-trained model, which is loaded without its top classification layer. The base of VGG16 is frozen to retain the learned features, and new fully connected layers are added for the classification task. The network structure is as follows:

**Base Model:** VGG16 (with include_top=False) to extract feature representations from images.

**Additional Layers:**
MaxPooling2D layer for downsampling.

Flatten to transform the 2D feature maps into 1D vectors.

Dense layers with increasing complexity: 1520 units, 750 units, 64 units, 32 units.

A Dropout layer (rate=0.5) to prevent overfitting.

The output layer consists of a single neuron with a sigmoid activation function for binary classification.


**Model Compilation and Training :**

The model is compiled with the binary crossentropy loss function and Adam optimizer with a learning rate of 0.0001. The model is trained for 10 epochs using the training generator for batches of 16 images at a time. The training process is monitored by evaluating both training accuracy and validation accuracy.


**Evaluation and Results :**

The model's performance is evaluated on the validation set, yielding the following results:

Test Loss: 0.2519

Test Accuracy: 93.95%

This indicates that the model is performing well with a high level of accuracy.


**Visualization :**

To understand the model's learning over time, two plots are generated to visualize:

**Training vs Validation Accuracy –** Displays how accuracy improves for both training and validation sets.

**Training vs Validation Loss –** Shows the loss trend for both sets, which is essential for detecting overfitting or underfitting.


**Model Deployment using Gradio:**

To make the model more accessible, a Gradio interface is created, allowing users to upload an image and receive a prediction (Fire or Non-Fire). The image is preprocessed (resized, normalized) before being passed to the model for classification. The output is a label indicating whether a forest fire is detected.


**Conclusion :**

The model successfully detects forest fires with an accuracy of 93.95% after 10 epochs. By leveraging transfer learning with the VGG16 pre-trained model and applying appropriate data augmentation and regularization techniques, the model is able to generalize well. The Gradio interface makes it easy for users to interact with the model and make predictions on new images.


In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
import numpy as np
import gradio as gr
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the train and validation data directories
train_data_dir = '/content/drive/My Drive/Deep Learning/forest_fire/Training and Validation'
validation_data_dir = '/content/drive/My Drive/Deep Learning/forest_fire/Testing'

# Image dimensions
img_width, img_height = 224, 224
batch_size = 16

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixels
    shear_range=0.2,  # Apply random shear transformations
    zoom_range=0.2,   # Apply random zoom
    horizontal_flip=True  # Flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load training data and augment it
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary'  # 'binary' for 2-class classification (fire and no-fire)
)

# Load validation data
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary'
)

# Verify class labels
print("Class indices (label mapping):", train_generator.class_indices)

Found 1520 images belonging to 2 classes.
Found 380 images belonging to 2 classes.
Class indices (label mapping): {'fire': 0, 'nofire': 1}


In [ ]:
# Load the pre-trained VGG16 model (without the top classification layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the base model layers
base_model.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# transfer model
model = Sequential()

model.add(base_model)
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# Flatten the output of the layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(1520, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Dropout layer for regularization
model.add(Dropout(0.5))

# Output layer with one neuron (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1520)                │       7,005,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 750)                 │       1,140,750 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │          48,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,911,295 (87.40 MB)

 Trainable params: 8,196,607 (31.27 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model
score = model.evaluate(validation_generator, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 1256s 13s/step - accuracy: 0.7945 - loss: 0.3975 - val_accuracy: 0.8816 - val_loss: 0.3422
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1207s 13s/step - accuracy: 0.9473 - loss: 0.1669 - val_accuracy: 0.9000 - val_loss: 0.2311
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1220s 13s/step - accuracy: 0.9645 - loss: 0.1059 - val_accuracy: 0.9421 - val_loss: 0.1730
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1222s 13s/step - accuracy: 0.9633 - loss: 0.1041 - val_accuracy: 0.9211 - val_loss: 0.2259
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1208s 13s/step - accuracy: 0.9754 - loss: 0.0928 - val_accuracy: 0.9421 - val_loss: 0.1540
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9771 - loss: 0.0886 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 6))  # 1 row, 2 columns

# Plotting accuracy and validation accuracy
axes[0].plot(history.history['accuracy'], label='Training accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation accuracy')
axes[0].set_title('Training and Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

# Plotting loss and validation loss
axes[1].plot(history.history['loss'], label='Training loss')
axes[1].plot(history.history['val_loss'], label='Validation loss')
axes[1].set_title('Training and Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
model.save("/content/drive/My Drive/Deep Learning/wildfire_model.keras")

In [ ]:
# Load your trained model (replace with the correct model path)
model = load_model(r"/content/drive/My Drive/Deep Learning/wildfire_model.keras")

# Function to preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Resize the image to the model's expected input size
    img_array = image.img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.  # Normalize pixel values to [0, 1]
    return img_array

# Function to make the prediction
def predict_fire(image):
    img_array = preprocess_image(image)  # Preprocess the input image
    prediction = model.predict(img_array)  # Make prediction using the model

    # Interpret the prediction (assuming binary classification)
    if prediction[0][0] > 0.5:
        return "Non-Fire"
    else:
        return "Fire"

# Create Gradio interface
interface = gr.Interface(
    fn=predict_fire,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title="Forest Fire Prediction",
    description="Upload an image to predict if it contains a forest fire or not."
)


# Launch the Gradio interface
interface.launch()